In [20]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [21]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [22]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [23]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [24]:
# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [25]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_reject = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()

In [30]:
train_item_ids = sales_train['item_id'].unique()
train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))


In [31]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [92]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [93]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [94]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')


In [95]:
a = training.groupby(['shop_id','date_block_num'])['shop_cnt_block'].sum()\
                        .reset_index()
a = a[a['shop_cnt_block'] == 0]
closed_shop_ids = a["shop_id"].unique()
#closed_shop_ids = np.setdiff1d(closed_shop_ids, [36])
print(closed_shop_ids)


#with pd.option_context('display.max_rows', 999):
    #print(a[a['item_cnt_block'] == 0])

training = training[~training['shop_id'].isin(closed_shop_ids)]

[ 9 17 20 27 29 30 33 36 40 43 51 54]


In [36]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
#columns = ["item_id", "shop_id", "item_category_id", "month", "shop_cat", "shop_item", "date_block_num"]
columns = ["item_id", "shop_id", "item_category_id"]




y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


KeyError: '[1048576, 1048578, 2097155, 786437, 262150, 262151, 1048584, 1835014, 1835018, 262156, 262157, 1572877, 1048600, 786460, 1310751, 1310754, 524323, 1310756, 262181, 1310757, 524324, 786466, 262185, 786468, 262188, 1048635, 61, 62, 1835072, 1835091, 1835094, 524381, 1572957, 524383, 524384, 524387, 1048676, 1835107, 1835108, 1667322, 524400, 524401, 524405, 2097272, 1572984, 2097275, 1048700, 262267, 126, 1310845, 128, 524413, 1048706, 524417, 1835153, 524434, 262291, 262292, 524435, 262294, 262295, 524436, 524437, 154, 1835154, 786580, 786586, 786588, 524461, 2097331, 1310900, 1310901, 524468, 1310904, 1835192, 2097339, 1310907, 1835197, 1835199, 1048780, 1048783, 1310928, 1048785, 786642, 1048788, 2097364, 524500, 786645, 1835222, 217, 2097369, 1835226, 231, 1310953, 235, 236, 1310957, 786670, 239, 786671, 786673, 242, 1573107, 786676, 524533, 524535, 262, 1310988, 262413, 1310989, 262418, 262419, 2097441, 291, 2097443, 293, 1835301, 297, 1835305, 1835306, 1311020, 786732, 1

In [210]:
training.head()

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,shop_cat,shop_item,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
0,30,2,24,0,40,14.0,987.0,15109.0,57.0,1,2_40,2_30,5.920290,12.705839,8.740981
1,30,2,25,0,40,14.0,907.0,12100.0,68.0,2,2_40,2_30,5.920290,12.705839,8.740981
2,30,2,26,0,40,5.0,762.0,13084.0,68.0,3,2_40,2_30,6.055427,12.517935,8.702927
3,30,2,27,0,40,4.0,859.0,10683.0,83.0,4,2_40,2_30,5.920290,12.705839,8.740981
4,30,2,28,0,40,5.0,843.0,9208.0,58.0,5,2_40,2_30,5.921986,12.423669,8.716483


In [96]:
def add_block_units_stats(df, cols, name):
    print(name)
    name_units = name + '_units'
    name_mean = name + '_mean'
    name_median = name + '_median'
    name_max = name + '_max'
    name_min = name + '_min'
    name_std = name + '_std'
    
    
    try:
        df.drop(columns=[name_units, name_mean, name_median],inplace=True)
    except:
        pass

    
    block_units = df.groupby(cols)[name].sum()\
                        .reset_index().rename(columns={name:name_units})
    df = df.merge(block_units, on=cols, how='left')
    df[name_units].fillna(0,inplace=True)
    df[name_units] = pd.to_numeric(df[name_units].astype(int),downcast='unsigned')
    del block_units
    
    block_units_med = df.groupby(cols)[name].median()\
                        .reset_index().rename(columns={name:name_median})
    print(block_units_med.head())
    df = df.merge(block_units_med, on=cols, how='left')
    df[name_median].fillna(0,inplace=True)
    #df[name_median] = pd.to_numeric(df[name_median].astype(int),downcast='unsigned')
    del block_units_med
    
    block_means = df.groupby(cols)[name].mean()\
                        .reset_index().rename(columns={name:name_mean})
    df = df.merge(block_means, on=cols, how='left')
    df[name_mean].fillna(0,inplace=True)
    #df[name_mean] = pd.to_numeric(df[name_mean],downcast='float')
    del block_means
    
    block_max = df.groupby(cols)[name].max()\
                        .reset_index().rename(columns={name:name_max})
    df = df.merge(block_max, on=cols, how='left')
    df[name_max].fillna(0,inplace=True)
    #df[name_max] = pd.to_numeric(df[name_max],downcast='float')
    del block_max
    
    block_min = df.groupby(cols)[name].min()\
                        .reset_index().rename(columns={name:name_min})
    df = df.merge(block_min, on=cols, how='left')
    df[name_min].fillna(0,inplace=True)
    #df[name_min] = pd.to_numeric(df[name_min],downcast='float')
    del block_min
    
    block_std = df.groupby(cols)[name].std()\
                        .reset_index().rename(columns={name:name_std})
    df = df.merge(block_std, on=cols, how='left')
    df[name_std].fillna(0,inplace=True)
    #df[name_std] = pd.to_numeric(df[name_std],downcast='float')
    del block_std
    
    gc.collect()
    return df


training = add_block_units_stats(training, ['item_id'], 'item_cnt_block')
training = add_block_units_stats(training, ['shop_id'], 'shop_cnt_block')
training = add_block_units_stats(training, ['item_category_id'], 'category_cnt_block')
training = add_block_units_stats(training, ['shop_id', 'item_category_id'], 'shop_category_cnt_block')
#training = add_block_units_stats(training, ['shop_id', 'item_id','date_block_num'], 'shop_item_block')

item_cnt_block
   item_id  item_cnt_block_median
0       30                    4.5
1       31                   12.5
2       32                   24.0
3       33                   15.5
4       38                    2.5
shop_cnt_block
   shop_id  shop_cnt_block_median
0        2                  832.5
1        3                  734.5
2        4                  896.0
3        5                 1073.0
4        6                 1775.0
category_cnt_block
   item_category_id  category_cnt_block_median
0                 0                        0.0
1                 2                      139.0
2                 3                      803.5
3                 5                       49.5
4                 6                      275.5
shop_category_cnt_block
   shop_id  item_category_id  shop_category_cnt_block_median
0        2                 0                             0.0
1        2                 2                             3.0
2        2                 3                          

In [238]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,item_cnt_block_units,item_cnt_block_median,item_cnt_block_mean,item_cnt_block_max,item_cnt_block_min,item_cnt_block_std,shop_cnt_block_units,shop_cnt_block_median,shop_cnt_block_mean,shop_cnt_block_max,shop_cnt_block_min,shop_cnt_block_std,category_cnt_block_units,category_cnt_block_median,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_min,category_cnt_block_std,shop_category_cnt_block_units,shop_category_cnt_block_median,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_min,shop_category_cnt_block_std
2019996,21123,50,30,0,61,0.0,1126.0,516.0,8.0,7,0,0.0,0.0,0.0,0.0,0.000000,53206400,1016.5,1040.0,1437.0,895.0,154.358886,49052072,654.0,787.1,2290.0,446.0,515.823768,11704,8.0,7.7,13.0,3.0,2.610457
1234286,13249,28,30,0,47,0.0,3612.0,630.0,45.0,7,0,0.0,0.0,0.0,0.0,0.000000,204645116,3766.5,4000.1,6879.0,2979.0,1036.085456,42005484,149.0,687.6,3421.0,0.0,1018.505680,40081,11.0,26.9,85.0,0.0,30.569874
377282,3989,14,26,0,56,0.0,1002.0,205.0,11.0,3,164,0.0,0.4,1.0,0.0,0.490496,53001760,1013.0,1036.0,1233.0,933.0,79.959521,2519122,192.0,198.2,312.0,85.0,74.355867,1488,5.0,4.8,11.0,1.0,3.492757
1797324,18653,46,28,0,40,0.0,1464.0,9208.0,156.0,5,82,0.0,0.2,1.0,0.0,0.400489,80438868,1558.0,1572.3,2100.0,1320.0,209.764797,3212026510,9293.5,10174.3,15109.0,6779.0,2444.994672,1364440,167.0,177.2,252.0,129.0,39.496362
1072000,11435,42,24,0,55,7.0,4125.0,9291.0,368.0,1,2091,5.0,5.1,8.0,2.0,1.702077,210103888,4129.0,4106.8,4648.0,3578.0,333.208239,1899903264,6413.5,6895.7,9291.0,4913.0,1348.041322,2184000,326.0,325.0,419.0,206.0,56.298061
1075831,11498,59,25,0,37,6.0,1206.0,4847.0,48.0,2,5535,10.0,13.5,28.0,3.0,9.426916,53671956,1023.0,1049.1,1293.0,790.0,160.835306,561635835,3908.5,4138.5,5959.0,2989.0,918.820587,179071,55.0,54.1,74.0,22.0,13.621529
1556002,16002,7,26,0,64,16.0,1430.0,1328.0,31.0,3,3567,4.0,8.7,36.0,0.0,10.634182,73609008,1333.5,1438.8,2084.0,1212.0,277.734167,61674496,991.0,1175.2,2211.0,801.0,424.519486,42112,32.0,32.9,61.0,12.0,13.951418
1712577,17714,2,31,0,49,1.0,942.0,571.0,5.0,8,1230,1.0,3.0,12.0,0.0,3.825663,43168808,832.5,843.8,987.0,727.0,77.605913,23335560,664.0,669.6,818.0,536.0,89.309960,5270,4.0,6.2,15.0,0.0,5.038836
1166302,12646,42,26,0,40,0.0,4133.0,13084.0,685.0,3,492,0.0,1.2,12.0,0.0,3.604398,210103888,4129.0,4106.8,4648.0,3578.0,333.208239,3212026510,9293.5,10174.3,15109.0,6779.0,2444.994672,4141830,496.0,537.9,702.0,430.0,92.699547
1614309,16227,21,33,3,65,48.0,1912.0,1518.0,53.0,10,35588,84.0,86.8,161.0,46.0,37.499196,93305608,1841.5,1823.8,2053.0,1622.0,124.217799,74668503,1879.0,2001.3,2941.0,1518.0,427.057020,47411,53.5,52.1,70.0,31.0,10.696529


In [239]:
training.describe()

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,item_cnt_block_units,item_cnt_block_median,item_cnt_block_mean,item_cnt_block_max,item_cnt_block_min,item_cnt_block_std,shop_cnt_block_units,shop_cnt_block_median,shop_cnt_block_mean,shop_cnt_block_max,shop_cnt_block_min,shop_cnt_block_std,category_cnt_block_units,category_cnt_block_median,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_min,category_cnt_block_std,shop_category_cnt_block_units,shop_category_cnt_block_median,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_min,shop_category_cnt_block_std
count,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06
mean,1.101830e+04,3.156098e+01,2.850000e+01,2.445384e-01,4.627815e+01,1.257451e+01,1.699154e+03,3.773882e+03,8.389024e+01,5.500000e+00,5.155550e+03,8.791634e+00,1.257451e+01,4.066224e+01,3.116302e+00,1.149800e+01,8.692870e+07,1.633695e+03,1.699154e+03,2.342293e+03,1.339854e+03,3.034243e+02,8.311411e+08,3.444695e+03,3.773882e+03,5.851549e+03,2.520990e+03,9.875358e+02,4.513030e+05,8.002772e+01,8.389024e+01,1.289736e+02,5.351564e+01,2.269051e+01
std,6.246577e+03,1.773250e+01,2.872282e+00,1.080989e+00,1.676899e+01,8.034849e+01,1.394110e+03,3.751384e+03,1.724458e+02,2.872282e+00,2.534712e+04,5.266674e+01,6.182224e+01,1.920728e+02,2.927392e+01,5.008119e+01,6.728722e+07,1.260525e+03,1.315231e+03,1.902227e+03,1.074559e+03,3.487790e+02,1.170224e+09,3.245611e+03,3.522610e+03,5.108302e+03,2.405506e+03,8.299934e+02,1.143970e+06,1.520227e+02,1.616670e+02,2.734789e+02,1.077633e+02,5.558359e+01
min,3.000000e+01,2.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,3.830000e+02,-1.000000e+00,-1.000000e+00,1.000000e+00,-4.100000e+01,0.000000e+00,-1.000000e-01,0.000000e+00,-1.000000e+00,0.000000e+00,2.178904e+07,4.220000e+02,4.259000e+02,4.840000e+02,3.830000e+02,3.450961e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,-4.000000e+00,0.000000e+00,-1.000000e-01,0.000000e+00,-1.000000e+00,0.000000e+00
25%,5.394250e+03,1.600000e+01,2.600000e+01,0.000000e+00,3.700000e+01,0.000000e+00,9.320000e+02,6.800000e+02,4.000000e+00,3.000000e+00,4.100000e+02,0.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,1.184662e+00,5.169718e+07,9.890000e+02,1.010500e+03,1.247000e+03,7.900000e+02,1.242178e+02,3.709315e+07,6.540000e+02,7.871000e+02,1.622000e+03,4.460000e+02,3.046296e+02,4.023000e+03,3.500000e+00,4.600000e+00,1.200000e+01,0.000000e+00,3.130728e+00
50%,1.119050e+04,3.400000e+01,2.850000e+01,0.000000e+00,4.300000e+01,2.000000e+00,1.272000e+03,2.203000e+03,3.300000e+01,5.500000e+00,1.722000e+03,2.000000e+00,4.200000e+00,1.100000e+01,0.000000e+00,3.138314e+00,6.666660e+07,1.318500e+03,1.303100e+03,1.715000e+03,9.900000e+02,1.843054e+02,1.802443e+08,1.917000e+03,1.880900e+03,4.824000e+03,1.316000e+03,9.188206e+02,4.686500e+04,3.150000e+01,3.400000e+01,6.000000e+01,1.700000e+01,1.218249e+01
75%,1.606725e+04,4.700000e+01,3.100000e+01,0.000000e+00,5.800000e+01,1.000000e+01,1.740000e+03,6.353000e+03,1.020000e+02,8.000000e+00,4.756000e+03,8.500000e+00,1.160000e+01,3.000000e+01,2.000000e+00,8.594192e+00,9.245635e+07,1.699500e+03,1.807200e+03,2.329000e+03,1.320000e+03,2.777342e+02,1.899903e+09,6.413500e+03,6.895700e+03,9.291000e+03,4.913000e+03,1.348041e+03,4.928000e+05,1.015000e+02,1.043000e+02,1.530000e+02,6.800000e+01,2.467247e+01
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,8.300000e+01,7.869000e+03,8.815000e+03,1.510900e+04,5.129000e+03,1.000000e+01,1.582846e+06,3.512000e+03

In [53]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'item_cnt_block', 'shop_cnt_block',
       'category_cnt_block', 'shop_category_cnt_block', 'month',
       'item_cnt_block_units', 'item_cnt_block_median',
       'item_cnt_block_mean', 'item_cnt_block_max', 'item_cnt_block_min',
       'item_cnt_block_std', 'shop_cnt_block_units',
       'shop_cnt_block_median', 'shop_cnt_block_mean',
       'shop_cnt_block_max', 'shop_cnt_block_min', 'shop_cnt_block_std',
       'category_cnt_block_units', 'category_cnt_block_median',
       'category_cnt_block_mean', 'category_cnt_block_max',
       'category_cnt_block_min', 'category_cnt_block_std',
       'shop_category_cnt_block_units', 'shop_category_cnt_block_median',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_max',
       'shop_category_cnt_block_min', 'shop_category_cnt_block_std'],
      dtype=object)

In [54]:
features = [
    
'item_cnt_block', 'shop_cnt_block',
       'category_cnt_block', 'shop_category_cnt_block', 
       
       'item_cnt_block_mean', 'item_cnt_block_max', 
       'item_cnt_block_std', 
       'shop_cnt_block_mean',
       'shop_cnt_block_max',  'shop_cnt_block_std',
       
       'category_cnt_block_mean', 'category_cnt_block_max',
       'category_cnt_block_std',
       
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_max',
        'shop_category_cnt_block_std'

]

In [97]:
training.fillna(0,inplace=True)

In [98]:
training[features].sample(10)

,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_cnt_block_mean,item_cnt_block_max,item_cnt_block_std,shop_cnt_block_mean,shop_cnt_block_max,shop_cnt_block_std,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_std,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_std
475045,52.0,3618.0,2984.0,119.0,41.4,89.0,27.606137,4106.8,4648.0,333.208239,3306.7,5620.0,968.485554,116.7,196.0,36.820185
429828,4.0,1432.0,4913.0,73.0,8.7,13.0,2.868810,1303.1,1664.0,204.059070,6895.7,9291.0,1348.041322,81.8,113.0,18.439376
1652477,1.0,1189.0,189.0,0.0,3.2,5.0,1.401710,1353.6,1715.0,229.996683,335.7,511.0,115.452394,0.0,0.0,0.000000
124454,40.0,979.0,1173.0,23.0,26.4,56.0,15.976939,1011.4,1248.0,127.271193,1142.9,1692.0,289.226620,25.7,39.0,8.104267
73595,0.0,385.0,417.0,1.0,2.1,5.0,1.642126,425.9,484.0,34.509611,579.5,1024.0,229.849294,3.0,9.0,2.531245
1057749,0.0,1061.0,1056.0,0.0,0.1,1.0,0.300367,1293.8,1747.0,181.719029,1575.7,4824.0,1200.883432,0.0,0.0,0.000000
1458993,2.0,1570.0,704.0,12.0,0.8,2.0,0.872845,1543.6,1993.0,200.016054,935.7,1681.0,304.629640,17.4,25.0,5.046493
138701,10.0,1779.0,8036.0,233.0,8.5,11.0,1.804978,1540.8,2108.0,236.274102,6895.7,9291.0,1348.041322,179.3,248.0,45.322353
1256499,0.0,742.0,3421.0,1.0,0.0,0.0,0.000000,848.0,1314.0,180.297075,687.6,3421.0,1018.505680,0.9,5.0,1.640673
1962766,0.0,1316.0,1411.0,34.0,4.0,35.0,10.453062,1411.7,2046.0,224.641929,1557.9,2432.0,404.149681,31.0,48.0,7.115221


In [99]:
from sklearn import preprocessing


training[features] = preprocessing.scale(training[features])


In [100]:
a = training[['shop_item_cnt_block'] + ['date_block_num','item_id', 'shop_id'] + features].sort_values(by=["date_block_num"]).groupby(["item_id", "shop_id"])#['item_id_mean_encoding']#[features+['item_cnt_block']]

In [101]:
#groups
lstm_data = []
lstm_y = []

for name, group in a:
    #print(group.values)
    steps = []
    ys = []
    #print("group.values",group.values)
    for step in group.values:
        #print("step", len(step))
        #step is np.array 
        #step[4:] is np.array print(type(step[4:]))
        steps.append(step[4:])
        #print(step[9])
        #print(step[0])
        ys.append(step[0])
    #remove last
    #print(type(steps[0:8]))
    lstm_data.append(np.array(steps[0:8]))
    #remove first
    #print(ys)
    lstm_y.append(ys[1:9][-1])
    
lstm_y = np.array([np.array([y]) for y in lstm_y])

In [58]:
np.array(np.array(lstm_data)).shape

(209756, 8, 16)

In [38]:
np.array(lstm_y).shape

(271148,)

In [102]:
small_data = np.array(lstm_data)
small_y = np.array(lstm_y)

In [103]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(small_data, small_y, test_size=0.33, random_state=42)

In [104]:
features

['item_cnt_block',
 'shop_cnt_block',
 'category_cnt_block',
 'shop_category_cnt_block',
 'item_cnt_block_mean',
 'item_cnt_block_max',
 'item_cnt_block_std',
 'shop_cnt_block_mean',
 'shop_cnt_block_max',
 'shop_cnt_block_std',
 'category_cnt_block_mean',
 'category_cnt_block_max',
 'category_cnt_block_std',
 'shop_category_cnt_block_mean',
 'shop_category_cnt_block_max',
 'shop_category_cnt_block_std']

In [112]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


#x_train_scaled = MinMaxScaler().fit_transform(x_train[features])


#x_reshaped = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 10, x_train_scaled.shape[1]))
    
#x_val_scaled_reshaped = np.reshape(x_val_scaled, (x_val_scaled.shape[0], 1, x_val_scaled.shape[1]))


callbacks = [
    EarlyStopping(monitor='val_loss', patience=1, verbose=0)
]


my_model = Sequential()
my_model.add(LSTM(units = 16,input_shape = (8,16)))
my_model.add(Dropout(0.5))
my_model.add(BatchNormalization())
my_model.add(Dropout(0.5))
#my_model.add(BatchNormalization())
#my_model.add(LSTM(units = 16))
#my_model.add(BatchNormalization())
#my_model.add(Flatten())
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()


#lstmd_dataa = np.array(np.array(lstm_data)).reshape(271148,8,51)
#print(lstmd_dataa.shape)
#print(lstmd_dataa)
#lstm_yy = np.array([np.array([y]) for y in lstm_y[0:100]])
#print(lstm_yy.shape)
#print(lstm_yy)

history = my_model.fit(train_data, y_train, batch_size=4096, epochs=100,
                      validation_data=(val_data,y_val), callbacks=callbacks
                      )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 16)                2112      
_________________________________________________________________
dropout_12 (Dropout)         (None, 16)                0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 16)                64        
_________________________________________________________________
dropout_13 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total params: 2,193
Trainable params: 2,161
Non-trainable params: 32
_________________________________________________________________
Train on 140536 samples, validate on 69220 samples
Epoch 1/100
140536/140536 [==============================] - 3s 24us/step - loss: 4.1566

In [63]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [68]:
features

['item_cnt_block',
 'shop_cnt_block',
 'category_cnt_block',
 'shop_category_cnt_block',
 'item_cnt_block_mean',
 'item_cnt_block_max',
 'item_cnt_block_std',
 'shop_cnt_block_mean',
 'shop_cnt_block_max',
 'shop_cnt_block_std',
 'category_cnt_block_mean',
 'category_cnt_block_max',
 'category_cnt_block_std',
 'shop_category_cnt_block_mean',
 'shop_category_cnt_block_max',
 'shop_category_cnt_block_std']

In [70]:
lstm_data[0]

array([[ 0.01774133, -0.51083057,  3.02158358, -0.15593446, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [ 0.01774133, -0.56821488,  2.21947946, -0.09214629, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [-0.09427076, -0.67222394,  2.4817827 , -0.09214629, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [-0.10671655, -0.60264547,  1.84175212, -0.00516242, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [-0.09427076, -0.61412233,  1

In [66]:
training_test = training[(training['shop_id'].isin(test['shop_id'].unique())) & (training['item_id'].isin(test['item_id'].unique()))]

In [ ]:
test_lstm_data = []

for index, row in test.iterrows():
    if index % 10000 == 0:
        print(index)
    line = training_test[(training_test['shop_id'] == row['shop_id']) & (training_test['item_id'] == row['item_id'])][features].values
    test_lstm_data.append(line[2:11].reshape(1,8,16))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000


In [87]:
row

ID         163200
shop_id        36
item_id      5037
Name: 163200, dtype: int64

In [88]:
training_test[training_test['shop_id'] == 36]

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,...,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_min,category_cnt_block_std,shop_category_cnt_block_units,shop_category_cnt_block_median,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_min,shop_category_cnt_block_std


In [72]:
x = training_test[(training_test['shop_id'] == 5) & (training_test['item_id'] == 5037)][features].values

In [85]:
my_model.predict(x[2:11].reshape(1,8,16))

array([[1.5130363]], dtype=float32)

In [84]:
x[2:11].reshape(1,8,16)

array([[[ 0.22931973, -0.4233195 , -0.00983155, -0.13273877,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 0.19198236, -0.46277121, -0.26520402, -0.21392371,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 0.92628386, -0.49289797, -0.10179763, -0.05735274,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 1.15030805, -0.5345016 , -0.07647364, -0.07474952,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 0.51557285

In [ ]:
training[['shop_item_cnt_block'] + ['date_block_num','item_id', 'shop_id'] + features].sort_values(by=["date_block_num"]).groupby(["item_id", "shop_id"])#['item_id_mean_encoding']#[features+['item_cnt_block']]

In [ ]:
features = [
    
'item_id_mean_encoding',
       'shop_id_mean_encoding', #'item_category_id_mean_encoding',
#       'month_mean_encoding', 'shop_cat_mean_encoding',
#       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
 
#       'item_block_units_rolling_3', 'item_block_mean_rolling_3',
#       'item_block_median_rolling_3', 'item_block_min_rolling_3',
#       'item_block_max_rolling_3', 'item_block_std_rolling_3',
    
#       'shop_block_units_rolling_3', 'shop_block_mean_rolling_3',
#       'shop_block_median_rolling_3', 'shop_block_min_rolling_3',
#       'shop_block_max_rolling_3', 'shop_block_std_rolling_3',
    
#       'cat_block_units_rolling_3', 'cat_block_mean_rolling_3',
#       'cat_block_median_rolling_3', 'cat_block_min_rolling_3',
#       'cat_block_max_rolling_3', 'cat_block_std_rolling_3',
    
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
    
    
#       'item_block_units_lag_1', 'item_block_mean_lag_1',
#       'item_block_median_lag_1', 'item_block_min_lag_1',
#       'item_block_max_lag_1', 'item_block_std_lag_1',
    
#       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
#       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
    
#      'cat_block_units_lag_1', 'cat_block_mean_lag_1',
#       'cat_block_median_lag_1', 'cat_block_min_lag_1',
#       'cat_block_max_lag_1', 'cat_block_std_lag_1',
    
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
#       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
#       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
    
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
#       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
#       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
    
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
    
    'shop_pred', 'shop_cat_pred', 'cat_pred'

]


In [60]:
features = ['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [40]:
from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [63]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor


lr_model =  BayesianRidge()
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

rmse:  1.0587979027152195


In [ ]:
lr_model.coef_

In [45]:
#test            = pd.read_csv('test.csv.gz')
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)

In [46]:
def predict(shop_id):
    return shop_models[shop_id].predict([[34]])[0][0]

#test['shop_pred'] = test.apply(lambda row: predict(row['shop_id']), axis=1)

def predict(shop_id, cat_id):
    if shop_id in shop_cat_models and cat_id in shop_cat_models[shop_id]:
        return shop_cat_models[shop_id][cat_id].predict([[34]])[0][0]

test['shop_cat_pred'] = test.apply(lambda row: predict(row['shop_id'],row['item_category_id']), axis=1)

def predict(cat_id):
    if cat_id in cat_models:
        return cat_models[cat_id].predict([[34]])[0][0]

test['cat_pred'] = test.apply(lambda row: predict(row['item_category_id']), axis=1)

In [47]:
item_features = [ 
    'item_id_mean_encoding'
                ]

merge_col = ['item_id']
cols=item_features+merge_col

test = test.merge(training.drop_duplicates('item_id')[cols], on=merge_col, how='left')

In [ ]:
shop_features = [
        'shop_id_mean_encoding'
]

merge_col = ['shop_id']
cols=shop_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
cat_features = [
        'item_category_id_mean_encoding'#,'cat_me_real'
]

merge_col = ['item_category_id']
cols=cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
shop_item_features = [
        'shop_item_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_item_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [53]:
shop_cat_features = [
        'shop_cat_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [48]:
def add_rolls_test(df, cols, name, rolls = [3]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name})\
            [cols+[roll_name]]
        
        print([cols[0:len(cols)-1]+[roll_name]])
        thirty_three = block_units_rolling_temp[block_units_rolling_temp['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1]+[roll_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')
    

        del block_units_rolling_temp
        gc.collect()
        

    
    return df
    

#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_units')
#test = add_rolls_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_median')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_min')
test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_max')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_std')

shop_cat_block_max 3
[['shop_id', 'item_category_id', 'shop_cat_block_max_rolling_3']]


In [ ]:
test = add_rolls_test(test, ['item_category_id','date_block_num'], 'cat_block_mean')

In [ ]:
test = add_rolls_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

In [49]:
def add_lags_test(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()
        
        thirty_three = result[result['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1] + [lag_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')

        gc.collect()
    
    return df
                                         

                                        
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_units')
test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_max')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_std')

test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')

test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

test = add_lags_test(test, ['item_id','date_block_num'], 'item_share_block')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_share_block')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'comp2')

shop_block_mean 1
shop_cat_block_units 1
shop_item_block_units 1
shop_item_block_mean 1
item_share_block 1


In [ ]:
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')

In [50]:
#training['comp2'] = training['item_share_block'] * training['shop_share_block']


test = add_lags_test(test, ['item_id','shop_id','date_block_num'], 'comp2')

comp2 1


In [54]:
test.fillna(0, inplace=True)

In [55]:
from sklearn.preprocessing import MinMaxScaler

test_scaled = MinMaxScaler().fit_transform(test[features])


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [64]:
preds = lr_model.predict(test_scaled)
preds.clip(0,20,out=preds)


array([ 0.        ,  0.        ,  0.15157035, ...,  0.26322811,
        0.28951454,  0.21382811])

In [92]:
preds = lg_model.predict(test[features])
preds.clip(0,20,out=preds)


array([ 0.11804522,  0.07073731,  0.18383498, ...,  0.04995914,
        0.28776269,  0.09947505])

In [96]:

print(np.mean(preds))
print(np.max(preds))

0.291327336924
17.0728554221


In [ ]:
preds = np.mean(np.array([lg_preds, lr_preds]),axis=0)

In [97]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [95]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)

In [ ]:
training.columns.values

In [30]:
all_features = ['item_id_mean_encoding',
       'shop_id_mean_encoding', 'item_category_id_mean_encoding',
       'month_mean_encoding', 'shop_cat_mean_encoding',
       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
       'shop_share', 'item_share', 'shop_pred', 'shop_cat_pred',
       'cat_pred']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier
clf = RandomForestRegressor(n_estimators=10, random_state=0, n_jobs=8)

# Train the classifier
clf.fit(x_train[all_features], y_train)

# Print the name and gini importance of each feature


In [ ]:
f_ = sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1][0:]

features = [f[0] for f in f_]

from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])


lr_model =  Ridget(alpha=0)
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

In [ ]:
sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1]

In [ ]:
all_features

In [38]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor
import math

all_f = all_features.copy()
keep = []

print(all_f)

best = 10000
while len(all_f) > 0:
    print("keep", keep)
    len_keep = len(keep)
    #print(len(all_f))
    res = {}
    for idx,feat in enumerate(all_f):

        potential = keep + [feat]

        x_train_scaled = MinMaxScaler().fit_transform(x_train[potential])
        x_val_scaled = MinMaxScaler().fit_transform(x_val[potential])


        lr_model =  Ridge(alpha=0)
        lr_model.fit(x_train_scaled, y_train)


        lr_val_preds = lr_model.predict(x_val_scaled)
        lr_val_preds.clip(0,20,out=lr_val_preds)
        rms = math.sqrt(mean_squared_error(y_val, lr_val_preds))

        res[feat] = rms
        
    sorted_by_value = sorted(res.items(), key=lambda kv: kv[1])
    best_score = sorted_by_value[0][1]
    best_feature = sorted_by_value[0][0]
    print("best_score", best_score)
    print("best_feature", best_feature)
    if best_score < best:
        best = best_score
        keep.append(best_feature)
        all_f.remove(best_feature)
    else:
        break
    
keep

['item_id_mean_encoding', 'shop_id_mean_encoding', 'item_category_id_mean_encoding', 'month_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_mean_encoding', 'date_block_num_mean_encoding', 'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3', 'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3', 'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3', 'shop_block_units_lag_1', 'shop_block_mean_lag_1', 'shop_block_median_lag_1', 'shop_block_min_lag_1', 'shop_block_max_lag_1', 'shop_block_std_lag_1', 'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1', 'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1', 'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1', 'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1', 'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1', 'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1', 'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1', 'shop_share', 'item_share', 'shop_pred

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return

best_score 1.2213909415288133
best_feature item_id_mean_encoding
keep ['item_id_mean_encoding']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were

...

best_score 1.0587356513519661
best_feature shop_item_block_mean_lag_1
keep ['item_id_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_block_units_lag_1', 'shop_cat_pred', 'shop_cat_block_units_lag_1', 'cat_pred', 'comp2_lag_1', 'shop_cat_block_max_rolling_3', 'shop_block_mean_lag_1', 'item_share_block_lag_1', 'shop_item_block_mean_lag_1']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

best_score 1.0587553332327528
best_feature shop_share_block_lag_1


['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [ ]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_by_value = sorted(x.items(), key=lambda kv: kv[1])[::-1]
sorted_by_value

In [91]:
gc.collect()
lgtrain = lgbm.Dataset(x_train[features], label=y_train)
lgval = lgbm.Dataset(x_val[features], label=y_val)


params = {
        "num_threads": 16,
        #"device": "gpu",
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        "max_bin": 30,#default 255
        #"num_leaves": 70, #default 31
        "bagging_fraction": 0.5,
        "bagging_freq": 1,
        "min_data_in_leaf": 2500,
        "feature_fraction": 0.7,
        #"lambda_l2": 3,
        #"max_depth": 7,
        #"min_gain_to_split": 10,
        "learning_rate" : 0.1,
        #"histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}

evals_result = {}
lg_model = lgbm.train(params, lgtrain, 20000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=10, 
                      verbose_eval=10, 
                      evals_result=evals_result)

scores = {}
for i,score in enumerate(lg_model.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's rmse: 1.16893
[20]	valid_0's rmse: 1.12071
[30]	valid_0's rmse: 1.10703
[40]	valid_0's rmse: 1.10052
[50]	valid_0's rmse: 1.09479
[60]	valid_0's rmse: 1.08897
[70]	valid_0's rmse: 1.08548
[80]	valid_0's rmse: 1.08165
[90]	valid_0's rmse: 1.07856
[100]	valid_0's rmse: 1.07572
[110]	valid_0's rmse: 1.07395
[120]	valid_0's rmse: 1.0719
[130]	valid_0's rmse: 1.07138
Early stopping, best iteration is:
[123]	valid_0's rmse: 1.07102


[('shop_cat_mean_encoding', 579),
 ('shop_cat_pred', 538),
 ('item_id_mean_encoding', 532),
 ('cat_pred', 525),
 ('shop_block_mean_lag_1', 398),
 ('shop_item_block_units_lag_1', 360),
 ('shop_cat_block_max_rolling_3', 333),
 ('shop_cat_block_units_lag_1', 307),
 ('shop_item_block_mean_lag_1', 118),
 ('item_share_block_lag_1', 0),
 ('comp2_lag_1', 0)]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor()

neigh.fit(x_train_scaled, y_train) 